<a href="https://colab.research.google.com/github/moveformyfuture/House_Price_Expectation/blob/main/House_Price_Expectation_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 머신러닝을 위한 패키지 설치
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install sklearn

In [5]:
# 패키지의 별칭을 적음(일반적인 약속)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [6]:
# 구글 드라이브에 접근할 수 있도록 아래 코드 입력
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# 데이터를 pandas의 DataFrame 형식으로 불러옴
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/집값예측 프로젝트/train.csv')
df

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
0,1,10,2392,Ex,3,968,Ex,2392,2392,Ex,2,2003,2003,2003,386250
1,2,7,1352,Gd,2,466,Gd,1352,1352,Ex,2,2006,2007,2006,194000
2,3,5,900,TA,1,288,TA,864,900,TA,1,1967,1967,1967,123000
3,4,5,1174,TA,2,576,Gd,680,680,TA,1,1900,2006,2000,135000
4,5,7,1958,Gd,3,936,Gd,1026,1026,Gd,2,2005,2005,2005,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,1346,6,1756,Gd,2,422,TA,872,888,Ex,2,1996,1997,1996,204000
1346,1347,9,2748,Gd,3,850,Ex,1850,1850,Ex,2,2006,2006,2006,390000
1347,1348,5,1214,TA,1,318,TA,1214,1214,TA,2,1967,1967,1967,143000
1348,1349,5,894,TA,2,440,TA,864,894,Gd,1,1974,1974,1974,131000


In [8]:
# 컬럼이 요약되지 않도록 설정
pd.set_option('display.max_columns', None) # 컬럼이 요약되지 않도록 설정합니다.
df

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
0,1,10,2392,Ex,3,968,Ex,2392,2392,Ex,2,2003,2003,2003,386250
1,2,7,1352,Gd,2,466,Gd,1352,1352,Ex,2,2006,2007,2006,194000
2,3,5,900,TA,1,288,TA,864,900,TA,1,1967,1967,1967,123000
3,4,5,1174,TA,2,576,Gd,680,680,TA,1,1900,2006,2000,135000
4,5,7,1958,Gd,3,936,Gd,1026,1026,Gd,2,2005,2005,2005,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,1346,6,1756,Gd,2,422,TA,872,888,Ex,2,1996,1997,1996,204000
1346,1347,9,2748,Gd,3,850,Ex,1850,1850,Ex,2,2006,2006,2006,390000
1347,1348,5,1214,TA,1,318,TA,1214,1214,TA,2,1967,1967,1967,143000
1348,1349,5,894,TA,2,440,TA,864,894,Gd,1,1974,1974,1974,131000


In [9]:
# 결측치 확인
def check_missing_col(dataframe) : 
    missing_col = []
    for col in dataframe.columns:
        missing_values=sum(dataframe[col].isna())
        is_missing=True if missing_values >=1 else False
        if is_missing:
            print('결측치가 있는 컬럼은 :{col}입니다')
            print('해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다')
            missing_col.append([col, dataframe[col].dtype])
        if missing_col==[]:
            print('결측치가 존재하지 않습니다.')
        return missing_col

missing_col=check_missing_col(df)

결측치가 존재하지 않습니다.


In [10]:
# 결측치 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1350 non-null   int64 
 1   Overall Qual    1350 non-null   int64 
 2   Gr Liv Area     1350 non-null   int64 
 3   Exter Qual      1350 non-null   object
 4   Garage Cars     1350 non-null   int64 
 5   Garage Area     1350 non-null   int64 
 6   Kitchen Qual    1350 non-null   object
 7   Total Bsmt SF   1350 non-null   int64 
 8   1st Flr SF      1350 non-null   int64 
 9   Bsmt Qual       1350 non-null   object
 10  Full Bath       1350 non-null   int64 
 11  Year Built      1350 non-null   int64 
 12  Year Remod/Add  1350 non-null   int64 
 13  Garage Yr Blt   1350 non-null   int64 
 14  target          1350 non-null   int64 
dtypes: int64(12), object(3)
memory usage: 158.3+ KB


In [11]:
# 텍스트 형식의 데이터를 숫자로 변환(Label-인코딩 방법)
num_df=df.copy()
num_df[num_df.columns[num_df.dtypes=='O']] = num_df[num_df.columns[num_df.dtypes=='O']].astype(str).apply(LabelEncoder().fit_transform)
# astype() : 데이터 타입 바꾸기, LabelEncoder() : 원 핫 인코딩
# 데이터 타입이 object인 columns만 선택해서 indexing = str로 데이터 타입을 바꾼 후 LabelEncoder() 함수를 적용

In [15]:
# 데이터 쪼개기
from sklearn.model_selection import train_test_split

# 데이터 설정
x=num_df.drop(['target'], axis=1) # df에서 target만 삭제
y=['target']

SyntaxError: ignored

In [25]:
y


In [23]:
# 다항회귀분석
from sklearn.linear_model import LinearRegression

pr = LinearRegression()                   # 선형회귀분석 객체 
pr.fit(X_train_poly, y_train)              # 학습하기 
X_test_poly = poly.fit_transform(X_test)   # X_test : 검증 데이터 . 2차항 형태로 변경 
r_square = pr.score(X_test_poly, y_test)   # 결정계수(R-제곱)
r_square                                   #  0.7087009262975685 
y_hat_test = pr.predict(X_test_poly)       # 2차항 형태로 변경된 테스트데이터 

NameError: ignored

In [24]:
# 다항회귀분석
from sklearn.linear_model import LinearRegression

pr = LinearRegression()                   # 선형회귀분석 객체 
pr.fit(X_train_poly, y_train)              # 학습하기 
X_test_poly = poly.fit_transform(X_test)   # X_test : 검증 데이터 . 2차항 형태로 변경 
r_square = pr.score(X_test_poly, y_test)   # 결정계수(R-제곱)
r_square                                   #  0.7087009262975685 
y_hat_test = pr.predict(X_test_poly)       # 2차항 형태로 변경된 테스트데이터 

NameError: ignored